### Jose Nieves Flores Maynez
### CS152
### Fall 2018

__Problem Definition:__ The problem that we are attempting to solve with the following program is how an Agent navigates in a world with imperfect information. Said Agent collects information from the neighboring cells and deduces the location of the possible dangers so that it can avoid them based on a set of rules that are given to it. This problem is interesting because it is a clear example of how we can teach an AI agent to take its own decisions if we give it a few pieces of information of what to expect and how to interpret it.

__Solution Specification:__ There were several steps to solving this problem. Firstly, we had to create the environment the Agent would be interacting with. Afterwards, we needed a way to pass the information from the Environment to the Agent. This was solved by accesing the maps for Pits and Wumpus that were included in the Board class. This is a very conservative AI that doesn't take any risks. This means that if there's a slight chance of falling to a pit or meeting the Wumpus by entering a room, it prefers to avoid it and look for another free room.
<br>
<br>
A very important step in solving this problem was to think of a way to represent the rules so that it would be able to apply them. This was done by using Propositional Logic. The values of the percepts were stored in lists of lists using Python but the info was accessed by calling the coordinates of the room you wanted information from.
<br>
<br>
Rule 1: Rule 1 is warning us of the possible existence of a Wumpus next to our current cell. This helps us identify which cells marked as available for exploring from the Pits are actually dangerous due to a possible Wumpus.
<br>
<br>
Rule 2: This rule determines the position of the Wumpus with some help from Rule 3. If there are 2 cells or a single cell with no information next to where we detect a stench, then the Wumpus must be in them. This is due to the original rule that states there is a stench in cells adjacent to X iff there was a Wumpus in X. Since our cell has a Stench, it must be in one of the cells we haven't investigated. If it's a single cell, we can shoot it and kill the Wumpus. This would be confirmed by its scream. 
Rule 2.5: If there are 2 cells, we can shoot at one to get information about it. We know a scream is heard iff the Wumpus is shot. We also knew that the Wumpus has to be in one of those 2 cells. Therefore, if a scream is heard, the Wumpus is dead and it was in front of us. Otherwise, the Wumpus was in the other cell.
<br>
<br>
Rule 3: Again, using Stench iff adjacent to Wumpus but we now use Modus Tollens to prove that No Stench, therefore no Wumpus, thanks to If Wumpus adjacent, then Stench.
<br>
<br>
Rule 4: The agent is also taught that its main goal is recovering the piece of gold as well as to associate the Glitter to te lingot. 
<br>
<br>
Rule 5: Rule 5 uses the same Modus Tollens Rule 3 did but applied to Breezes and Pits.
<br>
<br>
Another important part of the implementation was making sure that the agent got back to the exit in the safest and fastes way possible. To do so, an A* search was implemented. This was adapted from the A* code provided in Session 3.1 so that instead of trying to avoid the obstacles, the available next states are only places that were visited as they had been proven to be safe (since the Agent is still alive).

In [1]:
import random
import itertools

def get_random_coordinates(numbers,size = 1):
    # Generate all possible non-repeating pairs
    pairs = list(itertools.product(numbers, 2))

    # Randomly shuffle these pairs
    random.shuffle(pairs)
    return pairs[0:size]

In [3]:
import random
import itertools
import numpy as np


def get_random_coordinates(rows, columns, size = 1):
    # Generate all possible non-repeating pairs
    pairs = list(itertools.product(range(4),range(4)))

    # Remove our starting coordinate 
    # so we won't start in a wumpus nor hole
    pairs.remove((0,0))
    
    # Randomly shuffle these pairs
    random.shuffle(pairs)
    return pairs[0:size]


def get_adjacent(coord,maxrow,maxcol):
        y,x = coord
        adjacent = []
        if y<maxrow and x<maxcol:
            adjacent.append((y+1,x+1))
            
        if y>0 and x<maxcol:
            adjacent.append((y-1,x+1))
            
        if y<maxrow and x>0:
            adjacent.append((y+1,x-1))
            
        if y>0 and x>0:
            adjacent.append((y-1,x-1))
            
        return adjacent

In [4]:
class AStarNode:
    # Class constructor
    def __init__(self,coord,fval,gval,parent=None):
        self.coord = coord
        self.fval = fval
        self.gval = gval
        self.parent = parent
        self.pruned = False

    # Comparison function based on f cost
    def __lt__(self,other):
        return self.fval < other.fval

    # Convert to string
    def __str__(self):
        return str(self.coord)
    




def directions_back(self):
    goal = [0,0]
    
    def hmanhattan(coord):
        x = abs(goal[0] - coord[0])
        y = abs(goal[1] - coord[1])
        return x + y
    
    # Start node
    start_node = AStarNode(start,hmanhattan(start),0)
    
    # Dictionary with current cost to reach all visited nodes
    costs_db = {start:start_node}
    
    # Frontier, stored as a Priority Queue to maintain ordering
    from queue import PriorityQueue
    
    frontier = PriorityQueue()
    frontier.put(start_node)
    
    # next moves
    moves_orth = ((1,0),(0,1),(-1,0),(0,-1))
    
    # Begin A* Search
    expansion_counter = 0
    
    while not frontier.empty():
        # Take the next available node from the priority queue
        cur_node = frontier.get()
    
        if cur_node.pruned:
            continue # Skip if this node has been marked for removal
    
        # Check if we are at the goal
        if cur_node.coord == goal: break
    
        # Expand the node in the orthogonal directions
        for m in moves_orth:
            next_coord = tuple(sum(x) for x in zip(cur_node.coord,m))
    
            # Can only move in this direction if there is no obstacle there
            if next_coord in self.visited:
                gval = cur_node.gval + 1 # Tentative cost value for child
    
                # If the child node is already in the cost database (i.e. explored)
                if next_coord in costs_db:
                    if costs_db[next_coord].gval > gval:
                        costs_db[next_coord].pruned = True # Mark existing value for deletion from frontier
                    else:
                        continue # ignore this child, since a better path has already been found previously.
    
                hval = hmanhattan(next_coord) # Heuristic cost from next node to goal
                next_node = AStarNode(next_coord,gval+hval,gval,cur_node) # Create new node for child
                frontier.put(next_node)
                costs_db[next_coord] = next_node #Mark the node as explored
    
        expansion_counter = expansion_counter + 1
    
    # Reconstruct the optimal path
    optimal_path = [cur_node.coord]
    while cur_node.parent:
        optimal_path.append((cur_node.parent).coord)
        cur_node = cur_node.parent
    optimal_path = optimal_path[::-1]





In [330]:
# Remember that with arrays, coordinates are (y,x)

class Board:
    
    def __init__(self, danger=3):
        self.rows = 4
        self.columns = 4
        # Generate boards to store info on Wumpus and Holes
        self.h_board = [["X" for i in range(self.columns)] for _ in range(self.rows)]
        self.w_board = [["X" for i in range(self.columns)] for _ in range(self.rows)]
        self.g_board = [["X" for i in range(self.columns)] for _ in range(self.rows)]
        
        
        self.gold = self.get_random_coordinates()
        self.g_board[self.gold[0][0]][self.gold[0][1]] = "G"
        
        # Generate the coordinates for holes and Wumpus
        self.holes = self.get_random_coordinates(danger)
        
        # Assign coordinate to Wumpus
        # Make sure there is a place for the Wumpus.
        if self.holes:
            self.wumpus = self.holes.pop()
            self.w_board[self.wumpus[0]][self.wumpus[1]] = "W"
        
        # Figure out where we'd smell stench
        self.stenches = self.get_adjacent(self.wumpus)
        for coord in self.stenches:
            r,c = coord
            self.w_board[r][c] = "S"
                
                
        self.breezes = []
        # Place the hole in the map
        for coord in self.holes:
            y,x = coord
            self.h_board[y][x] = "O"
            self.breezes.append(self.get_adjacent(coord))
        
        # Place the breezes in the map for 
        for lista in self.breezes:
            for coord in lista:
                r,c = coord
                # Have to make sure we don't replace a hole
                if self.h_board[r][c] == "X":
                    self.h_board[r][c] = "B"

    def get_random_coordinates(self, size = 1):
        # Generate all possible non-repeating pairs
        pairs = list(itertools.product(range(self.rows),range(self.columns)))
    
        # Remove our starting coordinate 
        # so we won't start in a wumpus nor hole
        pairs.remove((0,0))
        
        # Randomly shuffle these pairs
        random.shuffle(pairs)
        return pairs[0:size]

    def get_adjacent(self,coord):
        y,x = coord
        adjacent = []
            
        # Right
        if x<self.columns-1:
            adjacent.append((y,x+1))
        # Down
        if y<self.rows-1:
            adjacent.append((y+1,x))
        # Left  
        if x>0:
            adjacent.append((y,x-1))
        # Up
        if y>0:
            adjacent.append((y-1,x))
            
        return adjacent
                
    def h_show(self):
        for row in self.h_board:
            print(row)
            
    def w_show(self):
        for row in self.w_board:
            print(row)
            
    def g_show(self):
        for row in self.g_board:
            print(row)
            


class Explorer:
    """Represents agent in Wumpus world."""
    
    def __init__(self,show =True):
        self.show = show
        
        self.prev_loc = (-1,-1)
        self.loc = (0,0)
        self.score = 0
        self.has_gold = False
        self.has_arrow = True
        self.wumpus_alive = True
        self.wumpus_found = False
        self.wumpus_loc = False
        
        self.smelled_stenches = []
        self.no_wumpus_room = []
        self.front = (0,1)
        
        
        self.det_row = 4
        self.det_col = 4
        
        # The visited rooms.
        self.visited = [self.loc]
        
        # The rooms in the border not yet visited.
        self.frontier = []
        
        # The rooms in frontier that are safe.
        self.available = []
        self.world = Board()
        
        self.h_board = [["_" for i in range(self.world.columns)] for _ in range(self.world.rows)]
        self.w_board = [["_" for i in range(self.world.columns)] for _ in range(self.world.rows)]
        self.g_board = [["_" for i in range(self.world.columns)] for _ in range(self.world.rows)]
        
        
        
    def h_show(self):
        for row in self.h_board:
            print(row)
            
    def w_show(self):
        for row in self.w_board:
            print(row)
            
    def g_show(self):
        for row in self.g_board:
            print(row)
    
    def up(self):
        self.loc = 0
    
    def exit(self):
        return
    
    def perceive(self):
        """Returns the percepts according to current location.
        
        The order of percepts is Breeze, Stench, Glitter.
        They are each represented by their initial letter.
        If one of them is not perceived, then it returns
        an "X" in its place.
        
        Output:
            List of strings.
        """
        if self.loc not in self.visited:
            self.visited.append(self.loc)

        x,y = self.loc[0],self.loc[1]
        return [self.world.h_board[x][y],
                self.world.w_board[x][y],
                self.world.g_board[x][y]]
        
    def plan(self):
        """Put the world's info into the system.
        
        Take the info received by self.perceive() and store it.
        Picks up the gold if it is perceived. Shoots arrow if it can
        deduce where the Wumpus is from it. Applies Propositional
        Logic to determine which rooms are safe to enter.
        """
        x,y = self.loc[0], self.loc[1]
        
        percept = self.perceive()

        # Places information in Agent's map
        self.h_board[x][y] = percept[0]
        self.w_board[x][y] = percept[1]
        
        # This happens if we fall in a pit.
        if percept[0] == "O":
            print("You're dead")

        # This happens if we bump into the Wumpus without killing it.
        if percept[1] == "W" and self.wumpus_alive:
            print("You're dead")
        
        # Rule 1
        # Detects Wumpus in vicinity.
        if percept[1] == "S":
            # Adds to list of Wumpus-neighboring cells.
            self.smelled_stenches.append(self.loc)
            # Checks which cells could be the Wumpus's
            lista = self.get_adjacent(self.loc)
            unknown = []
            for yx in lista:
                y,x = yx[0],yx[1]
                # If we have no info on the cell, Wumpus could be there.
                if self.w_board[y][x] == "_":
                    unknown.append(yx)
            # If we have 2 cells or less with no info, we can shoot into one
            # and if it was just 1 cell, we're certain to kill it. If they were
            # 2 cells, then the Wumpus must be in the other cell.
            # Rule 2
            if len(unknown) <=2:
                print(unknown)
                self.wumpus_found= True
                self.face(unknown[0])
                if self.has_arrow:
                    if self.shoot() != "scream":
                        print("missed")
                        self.wumpus_loc = unknown[1]

        # If it doesn't detect a Stench it means there's no Wumpus nearby.
        # Therefore, all cells in vicinity that hadn't been travelled by
        # are marked as safe from the Wumpus.
        # Rule 3
        else:
            adj = self.get_adjacent(self.loc)
            for coord in adj:
                if coord not in self.visited:
                    y,x = coord[0],coord[1]
                    if self.w_board[y][x] == "_":
                        self.w_board[y][x] = "T"
                    if coord not in self.no_wumpus_room:
                        self.no_wumpus_room.append(coord)
        
        percepts = self.perceive()
        # Rule 4
        if percept[2] == "G":
            self.pick()
            return True
        
        possibilities = self.get_adjacent(self.loc)
        possibilities.reverse()
        for coord in possibilities:
            if coord not in self.visited:
                if coord in self.frontier:
                    self.frontier.remove(coord)
                self.frontier.append(coord)
                
        # Check for pits in the frontiers.
        for coord in (self.frontier):
            listb = self.get_adjacent(coord)
            for yx in listb:
                y,x = yx[0],yx[1]
                # We check for an "X" in squares adjacent to the
                # frontier because that makes them safe from pits.
                # Rule 5
                if self.h_board[y][x] == "X":
                    if coord in self.available:
                        self.available.remove(coord)
                    if coord not in self.visited:
                        self.available.append(coord)
                    break
        
        
        # We look for the Wumpus only if it's alive
        if not self.wumpus_found:
            if self.smelled_stenches:
                for coord in self.smelled_stenches:
                    lista = self.get_adjacent(coord)
                    for coo in lista:
                        if coo in self.available:
                            # There could be a Wumpus here so we avoid exploring.
                            if self.w_board[coo[0]][coo[1]] == "_":
                                print("removed")
                                print(coo)
                                self.available.remove(coo)
                        listb = self.get_adjacent(coo)
                        listb.remove(coord)
                        count = 0
                        for yx in listb:
                            y,x = yx[0],yx[1]
                            if self.w_board[y][x] =="_":
                                count +=1
                        if count==1:
                            # This coo must be where the Wumpus is.
                            self.wumpus_found = coo
        
        # Makes sure we never visit the Wumpus if we miss.
        if self.wumpus_loc in self.available:
            self.available.remove(self.wumpus_loc)
        return False






    
    def pick(self):
        """Pick up the gold."""
        if self.show == True:
            self.show_maps()
        x,y = self.loc[0], self.loc[1]
        if self.world.g_board[x][y]=="G":
            print("Got gold!")
            self.world.g_board[x][y]="X"
            self.has_gold =True
        self.score -= 1
    
    def get_adjacent(self,coord):
        """Find adjacent rooms.
        """
        y,x = coord
        adjacent = []
        
        # Right
        if x<self.det_col-1:
            adjacent.append((y,x+1))
        # Down
        if y<self.det_row-1:
            adjacent.append((y+1,x))
        # Left  
        if x>0:
            adjacent.append((y,x-1))
        # Up
        if y>0:
            adjacent.append((y-1,x))
            
        if self.front == (1,0):
            adjacent = adjacent[1:] + adjacent[:1]
            
        elif self.front == (0,-1):
            adjacent = adjacent[2:]+adjacent[:2]
            
        elif self.front == (-1,0):
            adjacent = adjacent[3:]+adjacent[:3]
            
        return adjacent
    
    def go_to(self, coord):
        """Take agent to target room.
        """
        if coord in self.get_adjacent(self.loc):
            self.face(coord)
            self.move()
        else:
            route = self.directions_to(coord)
            print("This is it")
            print(route)
            print(route.pop(0))
            while len(route)>0:
                next_sq = route.pop(0)
                self.face(next_sq)
                self.move()
                print("Moving")
        
    def go_out(self, coord):
        """Take agent to starting room.
        """
        route = self.directions_to(coord)
        print("This is it")
        print(route)
        print(route.pop(0))
        while len(route)>0:
            next_sq = route.pop(0)
            self.face(next_sq)
            self.move()
            print("Moving")

    
    def move(self):
        """Moves agent forward.
        """
        self.score -=1
        self.loc = (self.loc[0]+self.front[0],self.loc[1]+self.front[1])
        
    
    def directions_to(self,goal = (0,0)):
        """Get optimal route traversing only visited rooms.
        
        Output:
            optimal_path(list): Contains tuples with coordinates
                                of the route back to the exit.
        """
        
        def hmanhattan(coord):
            x = abs(goal[0] - coord[0])
            y = abs(goal[1] - coord[1])
            return x + y
        
        start = (self.loc[0],self.loc[1])
        # Start node
        start_node = AStarNode(start,hmanhattan(start),0)
        
        # Dictionary with current cost to reach all visited nodes
        costs_db = dict()
        costs_db = {start:start_node}
        
        # Frontier, stored as a Priority Queue to maintain ordering
        from queue import PriorityQueue
        
        frontier = PriorityQueue()
        frontier.put(start_node)
        
        # next moves
        moves_orth = ((1,0),(0,1),(-1,0),(0,-1))
        
        # Begin A* Search
        expansion_counter = 0
        
        while not frontier.empty():
            # Take the next available node from the priority queue
            cur_node = frontier.get()
            if cur_node.pruned:
                continue # Skip if this node has been marked for removal
        
            # Check if we are at the goal
            if cur_node.coord == goal: break
        
            # Expand the node in the orthogonal directions
            for m in moves_orth:
                next_coord = tuple(sum(x) for x in zip(cur_node.coord,m))
        
                # Can only move in this direction if it had been visited before.
                if next_coord in self.visited:
                    gval = cur_node.gval + 1 # Tentative cost value for child
        
                    # If the child node is already in the cost database (i.e. explored)
                    if next_coord in costs_db:
                        if costs_db[next_coord].gval > gval:
                            costs_db[next_coord].pruned = True # Mark existing value for deletion from frontier
                        else:
                            continue # ignore this child, since a better path has already been found previously.
        
                    hval = hmanhattan(next_coord) # Heuristic cost from next node to goal
                    next_node = AStarNode(next_coord,gval+hval,gval,cur_node) # Create new node for child
                    frontier.put(next_node)
                    costs_db[next_coord] = next_node #Mark the node as explored
        
            expansion_counter = expansion_counter + 1
        
        # Reconstruct the optimal path
        optimal_path = [cur_node.coord]
        while cur_node.parent:
            optimal_path.append((cur_node.parent).coord)
            cur_node = cur_node.parent
        optimal_path = optimal_path[::-1]
        return optimal_path
    
    def face(self,coord):
        """Makes Agent turn in that room's direction.
        """
        y,x = self.loc[0],self.loc[1]
        y1,x1 = coord[0],coord[1]
        new_front = (y1-y,x1-x)
        while new_front != self.front:
            self.turn_r()
        
    def turn_r(self):
        self.score -=1
        if self.front == (0,1):
            self.front = (1,0)
        elif self.front == (-1,0):
            self.front = (0,1)
        elif self.front == (0,-1):
            self.front = (-1,0)
        elif self.front == (1,0):
            self.front = (0,-1)

        
    
    def climb(self):
        """Climb out the cave exiting Wumpus world.
        """
        self.score -= 1
        if self.loc == (0,0):
            if self.has_gold:
                self.score +=1000
        
    def shoot(self):
        """Shoot arrow.
        """
        print("shoot")
        self.score -= 10
        self.has_arrow = False
        index = self.front.index(0)
        tentative_wumpus = [self.loc[0],self.loc[1]]
        tentative_wumpus[index-1] +=self.front[index-1]
        if list(self.world.wumpus) == tentative_wumpus:
            self.wumpus_alive = False
            print("*Scream*")
            return "scream"
        if self.show == True:
            self.show_maps()
    
    def explore(self):
        while not self.has_gold:
            # Only breaks if it finds gold.
            if self.plan():
                break
            self.show_maps()
            print(self.available)
            # Check there are safe rooms left to explore.
            if self.available:
                next_loc = self.available.pop()
                #self.go_to(next_loc)
                self.loc = (next_loc)
                self.visited.append(self.loc)
            # We also decide to get out if there's no possible moves without risk.
            else:
                print("Couldn't find gold")
                break
            
        self.go_out((0,0))
        self.climb()
        return self.score
        
    def show_maps(self):
        print("Currently at:"+ str(self.loc))
        print("      Pit map                     Wumpus map")
        for i in range(self.det_row):
            print(str(self.h_board[i])+"         "+str(self.w_board[i]))
        
        

In [332]:
e = Explorer()
e.world.h_show()
print("")
e.world.w_show()
print("")
e.world.g_show()
print("")
e.explore()
#e.show_maps()

['X', 'X', 'X', 'X']
['X', 'X', 'X', 'X']
['B', 'X', 'B', 'X']
['O', 'B', 'O', 'B']

['X', 'X', 'X', 'X']
['X', 'X', 'X', 'S']
['X', 'X', 'S', 'W']
['X', 'X', 'X', 'S']

['X', 'G', 'X', 'X']
['X', 'X', 'X', 'X']
['X', 'X', 'X', 'X']
['X', 'X', 'X', 'X']

Currently at:(0, 0)
      Pit map                     Wumpus map
['X', '_', '_', '_']         ['X', 'T', '_', '_']
['_', '_', '_', '_']         ['T', '_', '_', '_']
['_', '_', '_', '_']         ['_', '_', '_', '_']
['_', '_', '_', '_']         ['_', '_', '_', '_']
[(1, 0), (0, 1)]
Currently at:(0, 1)
      Pit map                     Wumpus map
['X', 'X', '_', '_']         ['X', 'X', 'T', '_']
['_', '_', '_', '_']         ['T', 'T', '_', '_']
['_', '_', '_', '_']         ['_', '_', '_', '_']
['_', '_', '_', '_']         ['_', '_', '_', '_']
Got gold!
This is it
[(0, 1), (0, 0)]
(0, 1)
Moving


995

### References

https://www.quora.com/How-do-you-create-random-nonrepetitive-pairs-from-a-list-in-Python

A\* Search Example for Robot Navigation
R. Shekhar
August 10, 2017

https://cocalc.com/projects/51717d2d-de20-498e-97f2-a07ae5227263/files/3.1%20%7C%20Peer%20Instruction%20/%20Simulation%20Breakouts%20%7C%20Group%20F.ipynb?session=default